In [ ]:
import pandas as pd
import timeit
import numpy as np
from datetime import datetime
from datetime import timedelta
from math import sqrt
from sklearn.metrics import mean_squared_error
import warnings
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.arima_model import ARIMA

dateparse = lambda dates: pd.datetime.strptime(dates, '%d.%m.%Y %H:%M:%S')
dataFrame = pd.read_csv(r"C:\Users\Paulo Cesar\Estudos\Data Science\Ajuste Media de Compra\datasets\VENDA-GAM-3.CSV", 
                        sep=";", parse_dates=['dt_venda'], index_col='dt_venda', date_parser=dateparse)

In [ ]:
def buscarDataAtual():
    data_atual = datetime.today()
    data_em_texto = '{}-{}-{}'.format(data_atual.year, data_atual.month, '01')
    return data_em_texto
    
def adicionarIndicesFaltantes(df):
    idx = pd.date_range(min(df.index), dataAtual, freq='MS')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    return df

def calcularErro(arrayDeDataFrame, dataAtual):
    for i, df in enumerate(arrayDeDataFrame):
        if (len(df) > 7):
            aVendida = df.iloc[-6:, 3].values
            j = 1
            while j <= 12:
                aMM = df.iloc[-8:-2, j+2].values
                erroMM = sqrt(mean_squared_error(aVendida, aMM))
                df.loc[dataAtual, 'MM'+str(j)] = erroMM
                j+=12
    
def calcularMediaMovel(arrayDeDataFrame):
    j = 1
    while j <= 12:
        arrayDeDataFrame[len(arrayDeDataFrame)-1]['MM'+str(j)] = arrayDeDataFrame[len(arrayDeDataFrame)-1]['qt_vendida'].rolling(j).mean()
        j += 1
    calcularErro(arrayDeDataFrame, dataAtual)
    dfR = calcularPrevisaoMediaMovel()
    return dfR


def calcularPrevisaoMediaMovel():
    resultados = []

    for i in range(0, len(arrayDeDataFrame)):
        modelo = min(dict(arrayDeDataFrame[i].iloc[-1, 4:]), key=dict(arrayDeDataFrame[i].iloc[-1, 4:]).get)
        qt_previsao = arrayDeDataFrame[i][min(dict(arrayDeDataFrame[i].iloc[-1, 4:]), key=dict(arrayDeDataFrame[i].iloc[-1, 4:]).get)][len(arrayDeDataFrame[i])-2]
        erro = arrayDeDataFrame[i][min(dict(arrayDeDataFrame[i].iloc[-1, 4:]), key=dict(arrayDeDataFrame[i].iloc[-1, 4:]).get)][len(arrayDeDataFrame[i])-1]
        dt_previsao = arrayDeDataFrame[i].index[-1]
        cod_mercadoria = arrayDeDataFrame[i].iloc[:,1].unique().max()
        cod_empresa = arrayDeDataFrame[i].iloc[:,0].unique().max()
        lista = [modelo, qt_previsao, dt_previsao, cod_mercadoria, cod_empresa, erro ]
        resultados.append(lista)
    
    dfR = pd.DataFrame(resultados,columns=['modelo', 'qt_previsao', 'dt_previsao', 'cd_mercadoria','cd_empresa', 'RMSE'])    
    return dfR

def suavizacaoExponencialSimples():
    y_hat_avg = pd.DataFrame(teste.copy())
    model_fit = SimpleExpSmoothing(np.asarray(train)).fit(smoothing_level=0.6,optimized=False)
    y_hat_avg['SES'] = model_fit.forecast(len(teste))
    plt.figure(figsize=(16,4))
    plt.plot(train, label='Train', linewidth=3, markersize=6, marker='o', )
    plt.plot(teste, label='Test', linewidth=3, markersize=6, marker='o' )
    plt.plot(y_hat_avg['SES'],linewidth=3, label='SES')
    plt.legend(loc='best')
    rmse_ses = sqrt(mean_squared_error(teste, y_hat_avg.SES));    
    
dataAtual = buscarDataAtual()
dataFrame = dataFrame[dataFrame.index != dataAtual]

In [ ]:
inicio = timeit.default_timer()

arrayMercadoria = dataFrame['cd_mercadoria'].unique()
arrayEmpresa = dataFrame['cd_empresa'].unique()
arrayDeDataFrame = []

for emp in arrayEmpresa:
    i = 0
    for i, mer in enumerate(arrayMercadoria):
        df = dataFrame[(dataFrame['cd_mercadoria'] == mer) &
                       (dataFrame['cd_empresa'] == emp)].sort_values(by=['dt_venda'], ascending=True)
        if len(df) > 0:
            df = adicionarIndicesFaltantes(df)
            arrayDeDataFrame.append(df)
            
            dfR = calcularMediaMovel(arrayDeDataFrame)
            
        arrayDeDataFrame[len(arrayDeDataFrame) - 1].fillna(0, inplace=True)

        

fim = timeit.default_timer()

print(f'Tempo de execução: {(fim - inicio) / 60} minutos')

In [ ]:
arrayDeDataFrame[127]